In [9]:
import numpy as np
import pandas as pd
import xlsxwriter
import requests as rq
from scipy import stats
import math
from secret_case import API_TOKEN

In [10]:
stocks = pd.read_csv('/Users/hashimkhan/Desktop/algo_trading_final/sp_500_stocks.csv')

In [12]:
#Function sourced from: https://stackoverflow.com/questions/312443/how-do-i-split-a-list-into-equally-sized-chunks
def chunks (lst, n):
    """Yield succisve n-sized chunks form lst"""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

In [15]:
my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

dataframes = []

for symbol_string in symbol_strings:
    batch_api_url = "https://yfapi.net/v6/finance/quote"
    
    querystring = {"symbols": symbol_string}
    
    headers = {
        'x-api-key': API_TOKEN
    }

    data = rq.get(batch_api_url, headers=headers, params=querystring).json()

    rows = []
    for result in data.get('quoteResponse', {}).get('result', []):
        
        symbol = result.get('symbol', np.nan)
        
        regular_market_price = float(result.get('regularMarketPrice', np.nan))

        trailing_pe = float(result.get('trailingPE', np.nan))


        
        rows.append([symbol, regular_market_price, trailing_pe, np.nan])

    dataframe = pd.DataFrame(rows, columns=my_columns)
    dataframes.append(dataframe)

final_dataframe = pd.concat(dataframes, ignore_index=True)
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,112.4500,29.437172,NaN
1,AAL,12.9700,3.395288,NaN
2,AAP,54.5800,9.426598,NaN
3,AAPL,170.4750,28.555279,NaN
4,ABBV,151.5200,31.176954,NaN
...,...,...,...,...
499,YUM,123.7150,24.992930,NaN
500,ZBH,111.9550,46.647915,NaN
501,ZBRA,236.6445,19.349510,NaN
502,ZION,33.8800,5.975309,NaN
